Loading the model in 

In [89]:
import torch
import pandas as pd
from torch import nn 
import joblib

from common_utils import MLPBuilder
from torch.utils.data import DataLoader, TensorDataset


In [90]:
# we trained the model with 8 features
model = MLPBuilder(no_features=8, layers=[512, 512, 512, 256])

model.load_state_dict(torch.load('models/Grid_based/DNN_model_[512, 512, 512, 256].pth'))

C:\Users\randa\AppData\Local\Temp\ipykernel_50660\3690747287.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('models/Grid_based/DNN_mode

<All keys matched successfully>

## Loading test data in 

In [91]:


train_file_path = './Data/15_Nov_Readings/combined_data.csv'
test_file_path = './Data/23_Dec_Readings/combined_data.csv'

model_path = './models/Grid_based'
scaler_X = joblib.load(model_path +'/scaler_X.pkl')

cleaned_df = pd.read_csv(train_file_path)

In [92]:
cleaned_df

,Tx_0 RSSI,Tx_0 Batt,Tx_0 Time,Tx_1 RSSI,Tx_1 Batt,Tx_1 Time,Tx_2 RSSI,Tx_2 Batt,Tx_2 Time,Tx_3 RSSI,...,Tx_5 Batt,Tx_5 Time,Tx_6 RSSI,Tx_6 Batt,Tx_6 Time,Tx_7 RSSI,Tx_7 Batt,Tx_7 Time,X_Coord,Y_Coord
0,-58,4.16,0.0,-65,4.13,0.0,-72,4.13,0.0,127,...,4.09,0.0,-81,4.12,0.0,-73,4.10,0.0,0,0
1,-59,4.11,21.0,-67,4.15,21.0,-72,4.11,20.0,-77,...,4.07,17.0,-80,4.12,15.0,-75,NaN,17.0,0,0
2,-59,4.14,38.0,-67,4.14,38.0,-73,NaN,40.0,-77,...,4.08,40.0,-79,4.13,38.0,-76,4.11,35.0,0,0
3,-60,4.16,57.0,-68,4.13,57.0,-73,4.12,56.0,-70,...,4.07,54.0,-83,4.13,50.0,-76,NaN,51.0,0,0
4,-60,4.16,74.0,-68,4.13,74.0,-73,4.11,72.0,-70,...,NaN,79.0,-81,4.14,78.0,-76,NaN,79.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,-74,NaN,357.0,-66,NaN,358.0,-67,4.07,357.0,-54,...,4.00,357.0,-76,4.11,352.0,-77,4.06,347.0,5,6
1275,-73,4.11,373.0,-67,4.10,368.0,-67,4.08,368.0,-54,...,4.00,376.0,-75,4.10,373.0,-77,4.07,367.0,5,6
1276,-73,4.11,394.0,-67,NaN,394.0,-68,NaN,398.0,-54,...,4.01,402.0,-75,4.11,397.0,-78,4.06,392.0,5,6
1277,-73,4.02,419.0,-67,4.11,415.0,-68,4.08,415.0,-53,...,4.01,419.0,-74,4.11,415.0,-78,4.06,409.0,5,6


In [93]:
batt_columns = [col for col in cleaned_df.columns if 'Batt' in col]
time_columns = [col for col in cleaned_df.columns if 'Time' in col]
columns_to_drop = batt_columns+time_columns
RSSI_columns = cleaned_df.drop(columns=columns_to_drop)


#Train the model without NAN values
# cleaned_df = RSSI_columns.dropna()

#Convert NaN values to 0
cleaned_df = RSSI_columns.fillna(0)

In [94]:
cleaned_df

,Tx_0 RSSI,Tx_1 RSSI,Tx_2 RSSI,Tx_3 RSSI,Tx_4 RSSI,Tx_5 RSSI,Tx_6 RSSI,Tx_7 RSSI,X_Coord,Y_Coord
0,-58,-65,-72,127,-74,-75,-81,-73,0,0
1,-59,-67,-72,-77,-72,-76,-80,-75,0,0
2,-59,-67,-73,-77,-70,-81,-79,-76,0,0
3,-60,-68,-73,-70,-72,-81,-83,-76,0,0
4,-60,-68,-73,-70,-71,-82,-81,-76,0,0
...,...,...,...,...,...,...,...,...,...,...
1274,-74,-66,-67,-54,-69,-66,-76,-77,5,6
1275,-73,-67,-67,-54,-69,-66,-75,-77,5,6
1276,-73,-67,-68,-54,-70,-65,-75,-78,5,6
1277,-73,-67,-68,-53,-70,-66,-74,-78,5,6


In [95]:
X = cleaned_df[['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']]
Y = cleaned_df[['X_Coord', 'Y_Coord']]

X_test_scaled = scaler_X.transform(X)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)


Y_test_array = Y.to_numpy()

Y_test_tensor = torch.tensor(Y.values, dtype=torch.float32)
Y_test_tensor_x = torch.tensor(Y_test_array[:, 0], dtype=torch.long)  # X_Coord
Y_test_tensor_y = torch.tensor(Y_test_array[:, 1], dtype=torch.long)  # Y_Coord

test_dataset = TensorDataset(X_test_tensor, Y_test_tensor_x, Y_test_tensor_y)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [96]:
def evaluate_model(model, dataloader, loss_fn):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    criterion_x = nn.CrossEntropyLoss()
    criterion_y = nn.CrossEntropyLoss()

    with torch.no_grad():  # No need to compute gradients during testing
        for inputs, targets_x, targets_y in dataloader:  # 
            # Move data to the appropriate device
            inputs, targets_x, targets_y = inputs, targets_x, targets_y  # Split target coordinates

            # Forward pass
            logits_x, logits_y = model(inputs)  # Expecting two outputs

            # Compute loss for both X and Y predictions
            loss_x = criterion_x(logits_x, targets_x)
            loss_y = criterion_y(logits_y, targets_y)
            loss = loss_x + loss_y  # Combine both losses

            # Update running loss
            running_loss += loss.item()
            total += targets_x.size(0)  # Use targets_x to count samples

            # Calculate accuracy for classification
            preds_x = torch.argmax(logits_x, dim=1)
            preds_y = torch.argmax(logits_y, dim=1)
            correct += ((preds_x == targets_x) & (preds_y == targets_y)).sum().item()

    # Compute average loss and accuracy
    avg_loss = running_loss / len(dataloader)
    accuracy = correct / total

    return avg_loss, accuracy

In [97]:
from sklearn.model_selection import train_test_split


rssi_columns = ['Tx_0 RSSI', 'Tx_1 RSSI', 'Tx_2 RSSI', 'Tx_3 RSSI', 'Tx_4 RSSI', 'Tx_5 RSSI', 'Tx_6 RSSI', 'Tx_7 RSSI']


# for col in rssi_columns:
#     cleaned_df[col + '_blocked'] = (cleaned_df[col] == 127).astype(int)


X = cleaned_df[rssi_columns]  # Only the RSSI columns
# X_with_flags = pd.concat([X, cleaned_df[[col + '_blocked' for col in rssi_columns]]], axis=1)  # Add the blocked flags
Y = cleaned_df[['X_Coord', 'Y_Coord']]  # Target coordinates

# Step 4: Perform the train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



# Display the results
print("X_train:\n", X_train)
print("X_test:\n", X_test)
print("Y_train:\n", Y_train)
print("Y_test:\n", Y_test)

X_train:
       Tx_0 RSSI  Tx_1 RSSI  Tx_2 RSSI  Tx_3 RSSI  Tx_4 RSSI  Tx_5 RSSI  \
351         -68        -60        -64        -69        -74        -73   
673         -79        -69        -70        -58        -59        -65   
741         -80        -67        -79        -63        -73        -58   
781         -77        -74        -75        -59        -55        -71   
158         -68        -68        -70        -70        -71        -82   
...         ...        ...        ...        ...        ...        ...   
1044        -71        -60        -67        -65        -78        -71   
1095        -84        -71        -75        -59        -78        -73   
1130        -70        -69        -80        -64        -68        -71   
860         -69        -60        -64        -62        -68        -67   
1126        -70        -69        -79        -63        -69        -82   

      Tx_6 RSSI  Tx_7 RSSI  
351         -84        -75  
673         -63        -71  
741         -6

In [98]:
criterion = nn.MSELoss()
avg_loss, accuracy = evaluate_model(model,test_loader ,criterion)

print(f"Test Acc: {accuracy * 100:.2f}%, Test Loss: {avg_loss:.6f}\n")

Test Acc: 98.59%, Test Loss: 0.104753

